In [ ]:
# import block
import json
from time import sleep
import requests
import lxml
from bs4 import BeautifulSoup
from selenium import webdriver
import sqlite3 as lite
import pandas as pd
from pymongo import MongoClient

# Vacation Homework

## Q1. Python Basic, Web Scraping, RDBMS, SQL

### Q1-1

`Ancient Egyptian multiplication`(one of two multiplication methods used by scribes, was a systematic method for multiplying two numbers that does not require the multiplication table, only the ability to multiply and divide by 2, and to add.) 을 Python 으로 구현하세요.

example output)

```
enter two numbers: 24 15
  24      15 struck
  12      30 struck
   6      60 struck
   3     120 keep
   1     240 keep
The result is 360
```

In [76]:
# Q1-1
def get_action_count(value:int) -> int:
    """Value 를 2 로 나누었을 때, 몫이 0 이 되기까지의 횟수를 찾는 함수.
    return count
    """
    count = 1
    while (True):
        value //= 2        
        count += 1
    
        if value == 1:
            break
            
    return count

a, b = map(int, input("enter two numbers: ").split())
count = get_action_count(a)
result = 0

text1 = "{0:>" + str(count) + "}"
text2 = "{0:>" + str(count) + "}"

# a 가 홀수라면 b 의 값을 더해주며, a 를 2로 더이상 나눌 수 없을 때까지 반복한다.
for _ in range(count):
    if a % 2 == 0:
        curr_status = "struck"
    else:
        curr_status = "keep"
        
    print(text1.format(a), "", text2.format(b), curr_status)
    
    if curr_status == "keep":
        result += b
        
    a //= 2
    b *= 2

print("The result is {}".format(result))

enter two numbers: 27 82
   27     82 keep
   13    164 keep
    6    328 struck
    3    656 keep
    1   1312 keep
The result is 2214


### Q1-2

Twitter의 `#asiancup` 에 대한 트윗 검색결과(https://twitter.com/search?q=%23asiancup)를 sqlite 를 활용한 database에 적재하세요.
(10회 스크롤 했을 때의 트윗 본문 전체(이미지 제외))

`Requirements: selenium, sqlite3`

- **Caution** : https://twitter.com/robots.txt 의 지침을 따르세요.

```
# Every bot that might possibly read and respect this file.
User-agent: *
Allow: /*?lang=
Allow: /hashtag/*?src=
Allow: /search?q=%23
Disallow: /search/realtime
Disallow: /search/users
Disallow: /search/*/grid

Disallow: /*?
Disallow: /*/followers
Disallow: /*/following

Disallow: /account/not_my_account
Disallow: /account/deactivated
Disallow: /settings/deactivated

Disallow: /oauth
Disallow: /1/oauth

Disallow: /i/streams
Disallow: /i/hello

# Wait 1 second between successive requests. See ONBOARD-2698 for details.
Crawl-delay: 1
```

In [ ]:
#Q1-2

## Q2. SQL, RDBMS

`products.db` 에 대하여 다음 문제를 해결하세요.

### Q2-1

`products.db`에 존재하는 모든 `table`과 `column`을 조회하세요.

In [ ]:
#Q2-1

### Q2-2

`Customer`의 `Country`가 "Germany" 이거나 `City`가 "London" 인 모든 주문의 주문양(`Quantity`)의 합을 조회하세요.

In [ ]:
#Q2-2

### Q2-3

월별 총 주문 횟수와 총 주문 갯수, 평균 주문 금액(소수점 2째 자리에서 반올림)을 조회하세요.
(총 주문 횟수를 기준으로 내림차순 정렬)

In [ ]:
#Q2-3

## Q3. noSQL, json, Web Scraping

### Q3-1

다음 금융 시가총액 페이지(http://finance.daum.net/domestic/market_cap) 에서 KOSPI200 상위 30개 종목에 대하여,
뉴스공시 탭의 종목뉴스 최신 100개를 가져와 mLab의 새로운 Collection을 생성하여 저장한 뒤, 모든 뉴스의 `title`, `keywords`, `createdAt` 값을 표현하세요.

`Requirements: requests, json, pymongo`

hint1: 
```python
headers = {"Host":"finance.daum.net"}
requests.get(url, headers=headers)
```

hint2: `some_collection.find(query, {'_id':0,'title':1,})`

<img src="./daum-finance-news.png" width="300">

In [ ]:
#Q3-1

### Q3-2
위(mLab Collection)에서 불러온 모든 데이터에 대하여 각 종목별 `keywords`와 그 갯수를 표현하는 `dictionary`를 생성하여 quotes 라는 리스트에 적재하세요.

`Requirements: json, pymongo`

ex)
```python
quotes=[
    {
    'code':'A005930',
    'keywords':{
        '2018년':1,
        '매출액':2,
        ...
        }
    },
...
]
```

In [ ]:
#Q3-2

## Q3-3
불러온 quotes에 대하여 상위 30개 종목 별 keyword 리스트의 갯수를 구하세요.
그리고, 키워드 별 출현 갯수를 카운트하여 dictionary에 저장하여 보여주세요

hint: 

```python
len_count = {}
for i in map(len, a_words):
	if i in len_count:
		len_count[i] += 1
	else:
		len_count[i] = 1
```

In [ ]:
#Q3-3